In [1]:
import numpy as np
np.random.seed(42)
import pandas as pd
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import time

In [2]:
# time.sleep(1000)

In [3]:
def get_metrics(y_pred, y_test, to_print=True):
    correct_labels = np.where(y_pred==y_test)[0]
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred,average='macro')
    recall = metrics.recall_score(y_test, y_pred,average='macro')
    f1score = metrics.f1_score(y_test, y_pred,average='macro')
    # rocscore = metrics.roc_auc_score(y_test, y_pred,average='micro',multi_class="ovo")
    confusion_matrix = metrics.confusion_matrix(y_test, y_pred)  
    classification_report = metrics.classification_report(y_test, y_pred)

    if to_print:
        print("Identified {} correct labels out of {} labels".format(len(correct_labels), y_test.shape[0]))
        print("Accuracy:",accuracy)
        print("Precision:",precision)
        print("Recall:",recall)
        print("F1 Score:",f1score)
        # print("ROC AUC Score:",rocscore)
        print("Confusion Matrix:\n", confusion_matrix)
        print("Classification_Report:\n", classification_report)

#     return (correct_labels, accuracy, precision, recall, confusion_matrix, classification_report)

In [4]:
def rfcfit(fname):
    print("*****************")
    print(fname)
    
    
    df = pd.read_csv(f"saved_fits/{fname}.csv")
    avgtime = df.dropna()['timetaken'].mean()
    failedobjs = len(df) - len(df.dropna())
    
    print(f"There are {failedobjs} objects for whom at least 1 passband's fit failed.")
    print(f"Average time taken for 1 full object is {avgtime} seconds per object for all 6 passbands")
    df = df.fillna(0)
    X = df[df.columns[3:]].values
    y = df["objclass"].values
    
    rfc = RandomForestClassifier(n_estimators=200,random_state=42)
    preds = cross_val_predict(rfc, X, y, cv=10)
    
    get_metrics(preds,y)
    f1score = metrics.f1_score(preds,y,average='macro')
    
    return avgtime, f1score, failedobjs

In [5]:
allparams = ["bazin","fred","alercev2myguess","alercev2alerceguess","alercev2withjohannr","alercev1"]
paramname = ["Bazin","FRED","Alercev2: Sid Guesses","Alercev2: Alerce Guesses","Alercev2: Using r=tfall/trise","Alercev1: Based on Villar"]
columnname = ["Parameterisation Used","Average Time per Object for all 6 passbands (s)",
              "Average F1 Score for RFC (n=150)", "Total objects (of 480) where fit failed (for ≥ 1 passband)"]

In [6]:
summarylist = []
for i, param in enumerate(allparams):
    res = list(rfcfit(param))
    summarylist.append([paramname[i]]+res)

*****************
bazin
There are 11 objects for whom at least 1 passband's fit failed.
Average time taken for 1 full object is 0.03585948584854133 seconds per object for all 6 passbands
Identified 274 correct labels out of 480 labels
Accuracy: 0.5708333333333333
Precision: 0.5558971035145829
Recall: 0.5708333333333333
F1 Score: 0.5609119551863987
Confusion Matrix:
 [[54  1  1  0  1  0  1  2]
 [ 1 37  7  3  1  4  2  5]
 [ 0  6 40  1  2  2  8  1]
 [ 0  6  1 25 14  9  4  1]
 [ 1  1  3  8 35  3  8  1]
 [ 1  2  9 11  6 19  9  3]
 [ 0  8 11  8  7 14 11  1]
 [ 0  3  2  1  0  1  0 53]]
Classification_Report:
               precision    recall  f1-score   support

         AGN       0.95      0.90      0.92        60
      SLSN-I       0.58      0.62      0.60        60
        SNII       0.54      0.67      0.60        60
        SNIa       0.44      0.42      0.43        60
   SNIa-91bg       0.53      0.58      0.56        60
       SNIax       0.37      0.32      0.34        60
       SNIb

# SUMMARY

In [7]:
summarydf = pd.DataFrame(data=summarylist,columns=columnname) 
summarydf.style.set_properties(align="right")

,Parameterisation Used,Average Time per Object for all 6 passbands (s),Average F1 Score for RFC (n=150),Total objects (of 480) where fit failed (for ≥ 1 passband)
0,Bazin,0.035859,0.560912,11
1,FRED,0.048322,0.477657,54
2,Alercev2: Sid Guesses,0.087899,0.472882,10
3,Alercev2: Alerce Guesses,0.109991,0.471424,10
4,Alercev2: Using r=tfall/trise,0.115024,0.457674,13
5,Alercev1: Based on Villar,0.048615,0.500113,0


<!-- |Parameterisation Used| Average Time per Object for all   6 passbands (s) | Average F1 Score for RFC (n=150) | Total objects (of 480) where fit failed   (for ≥ 1 passband) |   |
|:----------------------:|:---------------------------------------------------:|:----------------:|:------------------------------------------------------:|---|
| Bazin                  | 0.0309                                              | 0.56 ± 0.02      | 11                                                     |   |
| FRED                   | 0.0452                                              | 0.47 ± 0.02      | 54                                                    |   |
| Alerce: Sid Guesses    | 0.0789                                              | 0.47 ± 0.02      | 10                                                     |   |
| Alerce: Alerce Guesses | 0.0947                                              | 0.47 ± 0.02      | 10                                                     |   |
| Alerce: Using R        | 0.0952                                              | 0.45 ± 0.02      | 13                                                     |   |
| Villar                 | 0.0375                                              | 0.50 ± 0.02      | 0                                                      |   | -->